In [1]:
from qdrant_client import QdrantClient, models

In [2]:
client = QdrantClient(url="http://localhost:6333")

In [3]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2, 3, 4, 5, 6],
        payloads=[
            {"color": "red", "city": "London",},
            {"color": "green", "city": "Moscow"},
            {"color": "blue", "city": "Moscow"},
            {"color": "green", "city": "London"},
            {"color": "red", "city": "Berlin"},
            {"color": "blue", "city": "London"},
        ],
        vectors=[
            [0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2],
            [0.3, 0.3, 0.3],
            [0.4, 0.4, 0.4],
            [0.5, 0.5, 0.5],
            [0.6, 0.6, 0.6],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [4]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        # When using must, the clause becomes true only if every condition listed inside must is satisfied.
        must=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(value="London"),
            ),
            models.FieldCondition(
                key="color",
                match=models.MatchValue(value="red"),
            ),
        ]
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None)],
 None)

In [5]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        # When using should, the clause becomes true if at least one condition listed inside should is satisfied. 
        should=[
            models.FieldCondition(
                key="city",
                match=models.MatchValue(value="London"),
            ),
            models.FieldCondition(
                key="color",
                match=models.MatchValue(value="red"),
            ),
        ]
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'Berlin', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)

In [6]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        # must_not is equivalent to the expression (NOT A) AND (NOT B) AND (NOT C).
        must_not=[
            models.FieldCondition(key="city", match=models.MatchValue(value="London")),
            models.FieldCondition(key="color", match=models.MatchValue(value="red")),
        ]
    ),
)

([Record(id=2, payload={'city': 'Moscow', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'Moscow', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)